In [1]:
import pandas as pd
import connect_tools as ct
import numpy as np
from itertools import combinations

In [2]:
def champions_stats():
    """Create a database with statistics of champions for each position."""
    password = ""
    connexion = ct.Connect_Tools(password=password)
    df_matches = connexion.read_db(table_name="matches")
    df_champions= connexion.read_db(table_name="champions_list")

    def c_c(column : str):
        x = df_matches[column]
        y = df_champions["name_champ"]
        compte = []
        for i in y:
            co =  x.eq(i).sum()
            compte.append(co)
        return compte
    
    def c_av(column : str):
        x = df_matches[column]
        w = df_matches["win_side"]
        y = df_champions["name_champ"]
        av_win = []
        for i in y:
            loc = x.eq(i)
            ind = loc[loc].index
            av = w.loc[ind].mean()
            if "red" in column:
                av = 1-av
            else:
                pass
            av_win.append(av)
        return av_win

    parametres = {"Champions": df_champions["name_champ"], "Red Top" : c_c("red_top"), "Red Top Mean" : c_av("red_top"),
                "Red Jungle" : c_c("red_jun"), "red Jungle Mean" : c_av("red_jun"), "Red Mid" : c_c("red_mid"),
                "Red Mid Mean" : c_av("red_mid"), "Red Bot" : c_c("red_bot"), "Red Bot Mean" : c_av("red_bot"),
                "Red Sup" : c_c("red_sup"), "Red Sup Mean" : c_av("red_sup"),
                "Blue Top" : c_c("blu_top"), "Blue Top Mean" : c_av("blu_top"),
                "Blue Jungle" : c_c("blu_jun"), "Blue Jungle Mean" : c_av("blu_jun"), "Blue Mid" : c_c("blu_mid"),
                "Blue Mid Mean" : c_av("blu_mid"), "Blue Bot" : c_c("blu_bot"), "Blue Bot Mean" : c_av("blu_bot"),
                "Blue Sup" : c_c("blu_sup"), "Blue Sup Mean" : c_av("blu_sup")
                }

    df_stats = pd.DataFrame(parametres)
    connexion.insert_db(dataframe = df_stats, table_name = "champions_stats")
    connexion.close_conn()

In [3]:
champions_stats()

In [4]:
password = ""
connexion = ct.Connect_Tools(password=password)

df_stats = connexion.read_db(table_name="champions_stats")
print(df_stats)

    Champions  Red Top  Red Top Mean  Red Jungle  red Jungle Mean  Red Mid  \
0      Aatrox     2104      0.503327           2         0.500000       50   
1        Ahri       24      0.541667           0              NaN     2602   
2       Akali      484      0.524793           1         0.000000     1160   
3      Akshan      214      0.570093           1         1.000000      684   
4     Alistar       14      0.571429           2         0.000000        8   
..        ...      ...           ...         ...              ...      ...   
162      Zeri       26      0.500000           1         1.000000       17   
163     Ziggs        0           NaN           0              NaN      105   
164    Zilean       35      0.628571           0              NaN       62   
165       Zoe       10      0.400000           0              NaN      323   
166      Zyra        2      0.500000         112         0.535714       11   

     Red Mid Mean  Red Bot  Red Bot Mean  Red Sup  ...  Blue To

In [7]:
df_stats.head()

,Champions,Red Top,Red Top Mean,Red Jungle,red Jungle Mean,Red Mid,Red Mid Mean,Red Bot,Red Bot Mean,Red Sup,...,Blue Top,Blue Top Mean,Blue Jungle,Blue Jungle Mean,Blue Mid,Blue Mid Mean,Blue Bot,Blue Bot Mean,Blue Sup,Blue Sup Mean
0,Aatrox,2104,0.503327,2,0.5,50,0.480000,1,1.000000,2,...,2452,0.464927,4,0.5,43,0.488372,1,1.000000,4,0.500000
1,Ahri,24,0.541667,0,NaN,2602,0.556879,3,0.000000,7,...,29,0.620690,0,NaN,2581,0.495544,3,0.333333,7,0.571429
2,Akali,484,0.524793,1,0.0,1160,0.522414,0,NaN,1,...,494,0.441296,0,NaN,1069,0.479888,0,NaN,0,NaN
3,Akshan,214,0.570093,1,1.0,684,0.546784,31,0.483871,4,...,188,0.505319,0,NaN,562,0.501779,32,0.437500,5,0.600000
4,Alistar,14,0.571429,2,0.0,8,0.625000,0,NaN,988,...,11,0.363636,2,0.0,12,0.666667,0,NaN,953,0.488982


In [9]:
print(df_stats["Red Top"].sort_values())

29        0
163       0
52        0
134       0
124       0
       ... 
106    1275
32     1333
23     1507
38     1662
0      2104
Name: Red Top, Length: 167, dtype: int64


In [5]:
def counter_stats():
    password = ""
    connexion = ct.Connect_Tools(password=password)
    df_matches = connexion.read_db(table_name="matches")
    df_matches.dropna(inplace=True)
    df_champions= connexion.read_db(table_name="champions_list")
    list_champ = df_champions["name_champ"].to_list()
    
    # Minoritization function applied on percent of game played for a champion. Create a representation of victory reliability.

    def fonction_log(x):
        k = -np.log(0.001)
        return 1 - np.exp(-k * x)
    
    # Verified match-up position of k and c in dataframe

    def verified_position(val, k, c):
        if val.index(k) in range(6,11):
            k_index = val.index(k)
            if val.index(c) == k_index - 5:
                return True
            else:
                return False
        else:
            return False

    # Create a dictionnary for counters dataframe
        
    b = "blu_"
    r = "red_"    
    columns_df_cs = {b + col: [] for col in list_champ}
    index_df_cs = {r + ind : [] for ind in list_champ}
    for col, position in columns_df_cs.items():
        position.append(index_df_cs)

    # Iterate all combinations of 2 champions, found correspondance in dataframe and return value for win side.
        
    for index, row in df_matches.iterrows():
        val = row.values
        for k , c in combinations(list_champ, 2):
            if k == c:
                columns_df_cs[b+k[r+c]].append(None)
            if k and c in val and verified_position(val, k, c) == True:
                columns_df_cs[b+k[r+c]].append(1)
                columns_df_cs[b+c[r+k]].append(0)
        for col, values in index_df_cs.items():
            if values == 0:
                columns_df_cs[k].append(0)
            else:
                dict_match_count = {col: len(valeur) for col, valeur in index_df_cs.items()}
                x = dict_match_count[col]/sum(dict_match_count.values())
                f_min = fonction_log(x)
                indice = sum(values)/dict_match_count[col] * f_min
            columns_df_cs[k].append(indice)
            
    df = pd.DataFrame(columns_df_cs, index= list_champ)
    return df

In [142]:
def counter_stats_test():
    password = ""
    connexion = ct.Connect_Tools(password=password)
    df_matches = connexion.read_db(table_name="matches")
    df_matches.dropna(inplace=True)
    df_matches = df_matches.iloc[0:100,:]
    df_champions= connexion.read_db(table_name="champions_list")
    list_champ = df_champions["name_champ"].to_list()
    
    # Minoritization function applied on percent of game played for a champion. Create a representation of victory reliability.

    def fonction_log(x):
        k = -np.log(0.001)
        return 1 - np.exp(-k * x)
    
    # Verified match-up position of k and c in dataframe

    def verified_position(val, k, c):
        if val.index(k) in range(6,11):
            k_index = val.index(k)
            c_index = val.index(c)
            if k_index - c_index == 5 and val[11] == 1:
                return 1
            elif k_index - c_index == 5 and val[11] == 0:
                return 0
            else:
                return False
        else:
            return False

    # Create a dictionnary for counters dataframe
        
    b = "blu_"
    r = "red_"    
    columns_df_cs = {b + col: [] for col in list_champ}
    for col, position in columns_df_cs.items():
        index_df_cs = {r + ind + "_" + col : [] for ind in list_champ}
        position.append(index_df_cs)

    # Iterate all combinations of 2 champions, found correspondance in dataframe and return value for win side.
        
    for index, row in df_matches.iterrows():
        val = list(row.values)
        print(val)
        for k in list_champ:
            for c in list_champ:
                if k == c:
                    columns_df_cs[b+k][0][r+c+"_"+b+k].append(None)
                    continue
                elif all(x in val for x in[k,c]) and verified_position(val, k, c) is not False:
                    vp = verified_position(val, k, c)
                    if vp == 1:
                        columns_df_cs[b+k][0][r+c+"_"+b+k].append(1)
                        print(vp)
                        print(k, c)
                    elif vp == 0:
                        columns_df_cs[b+k][0][r+c+"_"+b+k].append(0)
                        print(vp)
                        print(k, c)
                else:
                    continue
    return columns_df_cs

In [143]:
test = counter_stats_test()

[1, 'Urgot', 'Graves', 'Katarina', 'Lucian', 'Milio', 'Ornn', 'Volibear', 'Qiyana', 'Smolder', 'Morgana', 0, 6846391779, 15815]
0
Morgana Milio
0
Ornn Urgot
0
Qiyana Katarina
0
Smolder Lucian
0
Volibear Graves
[2, 'Yone', 'Fiddlesticks', 'Lucian', 'Twisted Fate', 'Rell', 'Urgot', 'Viego', 'Karma', 'Ashe', 'Blitzcrank', 1, 6846352896, 15815]
1
Ashe Twisted Fate
1
Blitzcrank Rell
1
Karma Lucian
1
Urgot Yone
1
Viego Fiddlesticks
[3, 'Irelia', 'Viego', 'Ekko', 'Smolder', 'Shaco', 'Urgot', 'Lee Sin', 'Yasuo', 'Ashe', 'Xerath', 0, 6846318698, 15815]
0
Ashe Smolder
0
Lee Sin Viego
0
Urgot Irelia
0
Xerath Shaco
0
Yasuo Ekko
[4, 'Vayne', 'Lee Sin', 'Akali', 'Draven', 'Bard', 'Urgot', "Bel'Veth", 'Syndra', 'Seraphine', 'Janna', 0, 6844382451, 15815]
0
Bel'Veth Lee Sin
0
Janna Bard
0
Seraphine Draven
0
Syndra Akali
0
Urgot Vayne
[5, 'Urgot', 'Lee Sin', 'Hwei', 'Jhin', 'Zilean', 'Darius', "Kha'Zix", 'Sylas', 'Aphelios', 'Soraka', 1, 6844313698, 15815]
1
Aphelios Jhin
1
Darius Urgot
1
Kha'Zix Lee S

In [144]:
print(test)

{'blu_Aatrox': [{'red_Aatrox_blu_Aatrox': [], 'red_Ahri_blu_Aatrox': [], 'red_Akali_blu_Aatrox': [], 'red_Akshan_blu_Aatrox': [], 'red_Alistar_blu_Aatrox': [], 'red_Amumu_blu_Aatrox': [], 'red_Anivia_blu_Aatrox': [], 'red_Annie_blu_Aatrox': [], 'red_Aphelios_blu_Aatrox': [], 'red_Ashe_blu_Aatrox': [], 'red_Aurelion Sol_blu_Aatrox': [], 'red_Azir_blu_Aatrox': [], 'red_Bard_blu_Aatrox': [], "red_Bel'Veth_blu_Aatrox": [], 'red_Blitzcrank_blu_Aatrox': [], 'red_Brand_blu_Aatrox': [], 'red_Braum_blu_Aatrox': [], 'red_Briar_blu_Aatrox': [], 'red_Caitlyn_blu_Aatrox': [], 'red_Camille_blu_Aatrox': [], 'red_Cassiopeia_blu_Aatrox': [], "red_Cho'Gath_blu_Aatrox": [], 'red_Corki_blu_Aatrox': [], 'red_Darius_blu_Aatrox': [], 'red_Diana_blu_Aatrox': [], 'red_Draven_blu_Aatrox': [], 'red_Dr. Mundo_blu_Aatrox': [], 'red_Ekko_blu_Aatrox': [], 'red_Elise_blu_Aatrox': [], 'red_Evelynn_blu_Aatrox': [], 'red_Ezreal_blu_Aatrox': [], 'red_Fiddlesticks_blu_Aatrox': [], 'red_Fiora_blu_Aatrox': [], 'red_Fizz_blu

In [82]:
password = ""
connexion = ct.Connect_Tools(password=password)
df_matches = connexion.read_db(table_name="matches")
df_matches.dropna(inplace=True)
df_matches = df_matches.iloc[0:1,:]
df_champions= connexion.read_db(table_name="champions_list")
list_champ = df_champions["name_champ"].to_list()

b = "blu_"
r = "red_"    
columns_df_cs = {b + col: [] for col in list_champ}
for col, position in columns_df_cs.items():
    index_df_cs = {r + ind + "_" + col : [] for ind in list_champ}
    position.append(index_df_cs)

for index, row in df_matches.iterrows():
    val = list(row.values)
    print(val)
    for k in list_champ:
        for c in list_champ:
            if k == c:
                columns_df_cs[b+k][0][r+c+"_"+b+k].append(None)
                continue
            elif all(x in val for x in[k,c]):
                print(k, c)
                continue
            else:
                print("not in")

[1, 'Urgot', 'Graves', 'Katarina', 'Lucian', 'Milio', 'Ornn', 'Volibear', 'Qiyana', 'Smolder', 'Morgana', 0, 6846391779, 15815]
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not in
not 